# TinyML ECG Classification

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

## Read Dataset

In [8]:
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
# !kaggle competitions download -c diabetes-classification
!kaggle datasets download -d shayanfazeli/heartbeat

 98% 97.0M/98.8M [00:05<00:00, 20.1MB/s]
100% 98.8M/98.8M [00:05<00:00, 19.7MB/s]


In [10]:
!unzip /content/heartbeat.zip

Archive:  /content/heartbeat.zip
  inflating: mitbih_test.csv         
  inflating: mitbih_train.csv        
  inflating: ptbdb_abnormal.csv      
  inflating: ptbdb_normal.csv        


In [11]:
df_train = pd.read_csv("/content/mitbih_train.csv", header=None)
df_test = pd.read_csv("/content/mitbih_test---.csv", header=None)
df_test_x = pd.read_csv("/content/testdata.csv")
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# df[187] is y (class 0~4)
print(df_train[187].value_counts())
print(df_test[187].value_counts())

0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: 187, dtype: int64
0.0    18103
4.0     1608
2.0     1448
1.0      556
3.0      162
Name: 187, dtype: int64


In [13]:
df_test[0][0] = 1.0

## Prepare Dataset

In [14]:
X_train = df_train.values[:, :-1]
y_train = df_train.values[:, -1].astype(int)

X_test  = df_test.values[:, :-1]
y_test  = df_test.values[:, -1].astype(int)

# testx,testy = df_test.values[:,-1]
testx,testy = df_test_x.values[:,:-1],df_test_x.values[:,-1].astype(int)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(testx.shape)
print(testy.shape)

(87554, 187)
(21877, 187)
(87554,)
(21877,)
(14, 187)
(14,)


In [15]:
# print(x_test.shape)
# print(y_test.shape)

NameError: ignored

In [ ]:
# y_test

array([0, 0, 0, ..., 4, 4, 4])

In [16]:
# reshape for Conv2D
x_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
x_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1, 1)
testx = testx.reshape(testx.shape[0],testx.shape[1],1,1)
print(x_train.shape)
print(x_test.shape)
print(testx.shape)

(87554, 187, 1, 1)
(21877, 187, 1, 1)
(14, 187, 1, 1)


In [17]:
# print(y_train.shape)
# print(y_test.shape)
# print(testy.shape)
# testy = testy.reshape(testy.shape[0],5)
# print(testy.shape)

(87554,)
(21877,)
(14,)


ValueError: ignored

In [18]:
from tensorflow.keras import utils
y_train = utils.to_categorical(y_train)
y_test  = utils.to_categorical(y_test)
testy = utils.to_categorical(testy,num_classes=5)
print(y_train.shape)
print(y_test.shape)
print(testy.shape)


(87554, 5)
(21877, 5)
(14, 5)


## Build Model

In [19]:
input_shape = x_train.shape[1:]
num_classes= 5

In [ ]:
# Build Model 
# from tensorflow.keras import models, layers
import keras 
from keras import models,layers
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, kernel_size=5, activation='relu', padding='same')(inputs)
#x = layers.Conv2D(16, kernel_size=5, activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2,2), padding='same')(x)

x = layers.Flatten()(x)
x = layers.Dense(16)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 187, 1, 1)]       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 187, 1, 16)        416       
                                                                 
 flatten_7 (Flatten)         (None, 2992)              0         
                                                                 
 dense_14 (Dense)            (None, 16)                47888     
                                                                 
 dense_15 (Dense)            (None, 5)                 85        
                                                                 
Total params: 48,389
Trainable params: 48,389
Non-trainable params: 0
_________________________________________________________________


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Flatten, Dense
smodel = Sequential()
smodel.add(InputLayer(input_shape=input_shape))
smodel.add(Conv2D(16, kernel_size=5, activation='relu', padding='same'))
smodel.add(Flatten())
smodel.add(Dense(16))
smodel.add(Dense(num_classes,activation='softmax'))

In [21]:
# Compile Model
smodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# # Compile Model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train Model

In [22]:
batch_size = 100
num_epochs = 10 

In [23]:
# Train Model
shistory = smodel.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(testx,testy))
# shistory = smodel.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(testx, testy))

Epoch 1/10
876/876 [==============================] - 15s 16ms/step - loss: 0.3502 - accuracy: 0.8986 - val_loss: 0.0856 - val_accuracy: 1.0000
Epoch 2/10
876/876 [==============================] - 17s 19ms/step - loss: 0.1942 - accuracy: 0.9477 - val_loss: 0.0575 - val_accuracy: 1.0000
Epoch 3/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1616 - accuracy: 0.9562 - val_loss: 0.0439 - val_accuracy: 1.0000
Epoch 4/10
876/876 [==============================] - 15s 17ms/step - loss: 0.1479 - accuracy: 0.9598 - val_loss: 0.0342 - val_accuracy: 1.0000
Epoch 5/10
876/876 [==============================] - 15s 17ms/step - loss: 0.1394 - accuracy: 0.9620 - val_loss: 0.0286 - val_accuracy: 1.0000
Epoch 6/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1337 - accuracy: 0.9635 - val_loss: 0.0259 - val_accuracy: 1.0000
Epoch 7/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1286 - accuracy: 0.9652 - val_loss: 0.0377 - val_accuracy:

In [ ]:
# Train Model
history = model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Epoch 1/10
876/876 [==============================] - 14s 16ms/step - loss: 0.3484 - accuracy: 0.9008 - val_loss: 0.2313 - val_accuracy: 0.9342
Epoch 2/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1936 - accuracy: 0.9476 - val_loss: 0.1843 - val_accuracy: 0.9511
Epoch 3/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1630 - accuracy: 0.9562 - val_loss: 0.1642 - val_accuracy: 0.9555
Epoch 4/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1486 - accuracy: 0.9598 - val_loss: 0.1587 - val_accuracy: 0.9568
Epoch 5/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1412 - accuracy: 0.9619 - val_loss: 0.1446 - val_accuracy: 0.9604
Epoch 6/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1351 - accuracy: 0.9631 - val_loss: 0.1457 - val_accuracy: 0.9623
Epoch 7/10
876/876 [==============================] - 14s 16ms/step - loss: 0.1303 - accuracy: 0.9645 - val_loss: 0.1371 - val_accuracy:

## Save Model

In [ ]:
# Save Model
models.save_model(model, 'ecg_cnn.hdf5')

In [ ]:
score = smodel.evaluate(x_test,y_test,verbose=2)

684/684 - 2s - loss: 0.1343 - accuracy: 0.9645 - 2s/epoch - 3ms/step
